<h1> Machine Learning w chmurze - Cloud ML </h1>

W tym notebooku pokażemy jak przenieść prosty model Tensorflow do GCP i uruchomić na nim predykcje

<h2> Predykcje na podstawie tekstu </h2>

<b>Źródło danych</b>: Yelp Restaurant Reviews (https://www.yelp.com/dataset/challenge)

Dataset zawiera między innymi informacje o restauracjach oraz opinie klientów

Zadaniem jest przewidzenie czy restauracje przejdą inspekcje (amerykańskiego) sanepidu na podstawie opinii gości oraz dodatkowych informacji takich jak lokalizacja i rodzaje kuchni serwowanych w restauracji.

<h2> Ustawienie zmiennych środowiskowych, import bibliotek </h2>

In [140]:
import warnings
warnings.filterwarnings('ignore')

In [141]:
BUCKET = 'dswbiznesie'
PROJECT = 'dswbiznesie'
REGION = 'europe-west1'
REPO = '/content/datalab/dswbiznesie'

In [142]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['REPO'] = REPO

In [143]:
%datalab project set -p $PROJECT

In [144]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [145]:
import google.datalab.ml as ml
import tensorflow as tf
import apache_beam as beam
import shutil
import datetime
from apache_beam.io.gcp.internal.clients import bigquery
import pandas as pd
import google.datalab.bigquery as bq
import numpy as np
import random
import string
import unicodedata
import sys
#print tf.__version__
import warnings
warnings.filterwarnings('ignore')

<h2> Dane źródłowe </h2>


Dataset pobrany ze strony Yelp zawiera następujące pliki

In [8]:
!gsutil -q ls -l gs://$BUCKET/rawdata/hygiene

 103941968  2017-11-06T00:52:39Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat
    831242  2017-11-03T01:14:18Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat.additional
    159046  2017-11-03T01:14:17Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat.labels
TOTAL: 3 objects, 104932256 bytes (100.07 MiB)


* <b>hygiene.dat</b>: każda linia zawiera połączone opinie klientów danej restauracji
* <b>hygiene.dat.labels</b>: dla pierwszych 546 linii przypisana jest dodatkowe pole w którym 0 oznacza to że restauracja przeszła inspekcje, 1 to że restauracja <b>nie</b> przeszła inspekcji. Reszta wpisów posiada wpis "[None]" i nie będą brane pod uwagę przy obliczaniu modelu
* <b>hygiene.dat.additional</b>: plik CSV gdzie w pierwszym polu znajduje się lista oferowanych rodzajów kuchnii, w drugim kod pocztowy restauracji który można uznać za przybliżenie lokalizacji restauracji. W trzecim polu znajduje się liczba opinii, w czwartym średnia ocena ( w skali 0-5, gdzie 5 oznacza ocene najlepszą).

<h2> Feature engineering używając Apache Beam i BigQuery</h2>

Dane źródłowe należy przetwrzyć i dostosować do postaci której będzie można łatwo użyć do uczenia i ewaluacji modelu. 
Najwygodniejszym choć nie najtańszym rozwiązaniem jest załadowanie danych do BigQuery.

Odpowiednim narzędziem do tego zadania jest Apache Beam i jego implementacja - Google Dataflow.
Job Dataflow uruchamiany jest w chmurze. Jego przebieg można śledzić w Konsoli GCP (https://console.cloud.google.com/dataflow).
Uruchomienie joba trwa powyżej minuty.

In [14]:
def create_record(rest_tuple):
    #print(rest_tuple)
    identity = rest_tuple[0]
    reviews = rest_tuple[1]['reviews_kv'][0]
    inspection_result = int(rest_tuple[1]['labels_kv'][0]) if rest_tuple[1]['labels_kv'][0] != "[None]" else None
    categories_temp = rest_tuple[1]['attributes_kv'][0].split("\"")
    categories = ",".join([ x.replace('\'', '').replace('[','').replace(']','').strip() for x in categories_temp[1].split(',')])
    #categories = categories_temp[1].replace('\'', '').replace('[','').replace(']','')
    attributes_temp = categories_temp[2].split(",")
    zip_code = attributes_temp[1]
    review_count = int(attributes_temp[2])
    avg_rating = float(attributes_temp[3])
    
    return { 
        'identity': identity, 
        'reviews': reviews,
        'inspection_result': inspection_result,
        'categories': categories,
        'zip_code': zip_code,
        'review_count': review_count,
        'avg_rating': avg_rating
    }

def preprocess(RUNNER,BUCKET,BIGQUERY_TABLE):
    job_name = 'hygiene-ftng' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
    print 'Launching Dataflow job {} ... hang on'.format(job_name)
    OUTPUT_DIR = 'gs://{0}/data/hygiene/'.format(BUCKET)
    options = {
        'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
        'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
        'job_name': 'hygiene-ftng' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
        'project': PROJECT,
        'region': 'europe-west1',
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True
    }
    opts = beam.pipeline.PipelineOptions(flags=[], **options)
    p = beam.Pipeline(RUNNER, options=opts)
    
    # Adding table definition
    table_schema = bigquery.TableSchema()
    
    # Fields definition
    identity_schema = bigquery.TableFieldSchema()
    identity_schema.name = 'identity'
    identity_schema.type = 'integer'
    identity_schema.mode = 'required'
    table_schema.fields.append(identity_schema)
    
    
    reviews_schema = bigquery.TableFieldSchema()
    reviews_schema.name = 'reviews'
    reviews_schema.type = 'string'
    reviews_schema.mode = 'required'
    table_schema.fields.append(reviews_schema)

    inspection_result_schema = bigquery.TableFieldSchema()
    inspection_result_schema.name = 'inspection_result'
    inspection_result_schema.type = 'integer'
    inspection_result_schema.mode = 'nullable'
    table_schema.fields.append(inspection_result_schema)
    
    categories_schema = bigquery.TableFieldSchema()
    categories_schema.name = 'categories'
    categories_schema.type = 'string'
    categories_schema.mode = 'required'
    table_schema.fields.append(categories_schema)
    
    zip_code_schema = bigquery.TableFieldSchema()
    zip_code_schema.name = 'zip_code'
    zip_code_schema.type = 'string'
    zip_code_schema.mode = 'required'
    table_schema.fields.append(zip_code_schema)
    
    review_count_schema = bigquery.TableFieldSchema()
    review_count_schema.name = 'review_count'
    review_count_schema.type = 'integer'
    review_count_schema.mode = 'required'
    table_schema.fields.append(review_count_schema)
    
    avg_rating_schema = bigquery.TableFieldSchema()
    avg_rating_schema.name = 'avg_rating'
    avg_rating_schema.type = 'float'
    avg_rating_schema.mode = 'required'
    table_schema.fields.append(avg_rating_schema)
    
    #processing logic
    reviews = p | 'Read Reviews' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat'.format(BUCKET))
    labels = p | 'Read Labels' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat.labels'.format(BUCKET))
    attributes = p | 'Read Attributes' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat.additional'.format(BUCKET))
    
    reviews_kv = reviews | 'Map Reviews to KV' >> beam.Map(lambda x: (x.split(",")[0], ",".join(x.split(",")[1:]).replace("|","")))
    labels_kv = labels | 'Map Labels to KV' >> beam.Map(lambda x: (x.split(",")[0], x.split(",")[1]))
    attributes_kv = attributes | 'Map Attributes to KV' >> beam.Map(lambda x: (x.split(",")[0], x))
    
    restaurants = (
        {'reviews_kv': reviews_kv, 'labels_kv': labels_kv, 'attributes_kv': attributes_kv}
        | 'CoGroup By Restaurant Key' >> beam.CoGroupByKey())
    
    records = restaurants | 'Create Records' >> beam.Map(create_record)
    
    records | 'Write to BigQuery' >> beam.io.Write(
        beam.io.BigQuerySink(
            BIGQUERY_TABLE,
            schema=table_schema,
            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
            write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE))
    return p.run()

<b> Uruchomienie etl </b>

In [19]:
bigquery_dataset = "{}:{}.hygiene".format(PROJECT,PROJECT)
#preprocess('DirectRunner',BUCKET, bigquery_dataset)
job = preprocess('Dataflow', BUCKET, bigquery_dataset)

Launching Dataflow job hygiene-ftng-171118-193842 ... hang on


<h2> Przygotowanie danych do trenowania modelu </h2>

<b> Pobranie danych do DataFrame (pandas) </b>

In [146]:
query="""
SELECT
  #identity,
  reviews,
  inspection_result ,
  categories,
  zip_code,
  review_count,
  avg_rating
FROM
  `dswbiznesie.hygiene`
WHERE
  inspection_result IS NOT NULL
"""

<b> Podzial danych na zestaw treningowy i ewaluacyjny </b>

In [150]:
traindf = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(reviews)),4) > 0").execute().result().to_dataframe()
evaldf  = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(reviews)),4) = 0").execute().result().to_dataframe()
traindf.head()

,reviews,inspection_result,categories,zip_code,review_count,avg_rating
0,I went here on a first date which was a tad u...,0,Restaurants,98101,9,3.111111
1,Not impressive. While the decor is pretty gar...,0,Restaurants,98101,3,2.666667
2,Please note that this review only stems from ...,0,"Irish,Restaurants",98101,37,3.302326
3,I grew up in the central valley of California...,0,"Mexican,Restaurants",98101,3,3.333333
4,I'm convinced this place needs to be on Guy F...,0,"Mexican,Restaurants",98101,15,3.200000


In [96]:
traindf['inspection_result'].value_counts()

0    205
1    203
Name: inspection_result, dtype: int64

In [97]:
evaldf['inspection_result'].value_counts()

1    70
0    68
Name: inspection_result, dtype: int64

In [151]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

def preprocess_text(text):
  cleanedtxt = text.replace("&#160;"," ").lower()
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(cleanedtxt)
  stop = stopwords.words('english')
  nostoptokens = [i for i in tokens if i not in stop]
  snowball = SnowballStemmer('english')
  stems = [snowball.stem(i) for i in nostoptokens]
  return ' '.join(stems)

In [155]:
traindf['reviews'] = traindf['reviews'].apply(lambda x: preprocess_text(x))
evaldf['reviews'] = evaldf['reviews'].apply(lambda x: preprocess_text(x))

In [156]:
traindf.to_csv('train.csv', header=False, index=False, encoding='utf-8', sep='|')
evaldf.to_csv('eval.csv', header=False, index=False, encoding='utf-8', sep='|')

In [157]:
!head -3 train.csv

went first date tad uncomfort anyway went 8 00pm saturday music realli loud amp techno ish probabl ok like total alon haha ok enough mess coupl martini stoli dirti oliv ok like 3 oliv 2 c mon stingi kettl one w lemon said water realli know tri bad place definit date place chair w tall back hard atmospher nice modern servic great drink ok ladi got dress went w hotel bar last friday night place definit swanki prepar come dress impress cool side note fantasia barrino sit back tabl entourag crew tri hard stare much drink pretti spendi luckili happi hr start 10 pm mon fri enjoy tasti app definit tri sloppi joe like pull pork slider warn may intrigu sexi fri noth sexi sexi fri breath devour like limit garlic fri intak game safeco field needless say bit paranoid rest even get close person anyon confus place peopl seem flock yes chic cozi insid hotel bar popular seattl total conveni bf tire ventur downtown decid give w bar tri sinc could stumbl back upstair need signatur cocktail defin pricey 

In [30]:
!wc -l *.csv

    138 eval.csv
    408 train.csv
    408 vocab.csv
    954 total


In [31]:
%bash
gsutil cp *.csv gs://${BUCKET}/data/

Copying file://eval.csv [Content-Type=text/csv]...
Copying file://train.csv [Content-Type=text/csv]...
Copying file://vocab.csv [Content-Type=text/csv]...
- [3 files][  7.6 MiB/  7.6 MiB]                                                
Operation completed over 3 objects/7.6 MiB.                                      


In [32]:
!gsutil -q ls -l gs://$BUCKET/data

         0  2017-11-03T00:08:16Z  gs://dswbiznesie/data/
   1242719  2017-11-18T19:42:33Z  gs://dswbiznesie/data/eval.csv
   3353088  2017-11-18T19:42:34Z  gs://dswbiznesie/data/train.csv
   3353088  2017-11-18T19:42:34Z  gs://dswbiznesie/data/vocab.csv
     39652  2017-11-11T18:14:43Z  gs://dswbiznesie/data/vocab_words
                                 gs://dswbiznesie/data/hygiene/
TOTAL: 5 objects, 7988547 bytes (7.62 MiB)


<h2> Model Tensorflow </h2>

Właściwy model tensorflow znajduje się w pliku <b>model.py</b> a definicja joba Cloud ML w pliku <b>task.py</b>
Kod poniżej ma za zadanie zilutrować działanie kodu tensorflow.

In [33]:
import tensorflow as tf
from tensorflow.contrib import lookup
from tensorflow.python.platform import gfile

print tf.__version__
MAX_DOCUMENT_LENGTH = 100000  
PADWORD = 'ZYXW'

# vocabulary
lines = ['This might be the best "taco" truck on the planet. Hidden between a smoke shop and The Home Depot, this semi permanent, stylish and clean cafe on wheels.', 
         'Im always looking for a good place to sing karaoke. This is one of them. Drinks are cheap. Food is delicious. And its laid back and fun. I shall return!', 
         'Crazy Man just Crazy there are like 3 different places called Saigon Deli at this intersection but this one is on Jackson just west of twelfthand with $2 pork Banh Mi you cant go wrong, I am going to Indiana for work so I went in and got 6 Pork', 
         'Ive eaten here a few times in the past and thought it was decent. I went last night, however, and our meal was really subpar so the place seems to have gone down hill.We had chow fun noodles and the hollow vegetables with chili sauce']

lines = lines.map(lambda x: preprocess_text(x))

# create vocabulary
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
vocab_processor.fit(lines)
with gfile.Open('vocab.tsv', 'wb') as f:
    f.write("{}\n".format(PADWORD))
    for word, index in vocab_processor.vocabulary_._mapping.iteritems():
      f.write("{}\n".format(word))
N_WORDS = len(vocab_processor.vocabulary_)
print '{} words into vocab.tsv'.format(N_WORDS)

# can use the vocabulary to convert words to numbers
table = lookup.index_table_from_file(
  vocabulary_file='vocab.tsv', num_oov_buckets=1, vocab_size=None, default_value=-1)
numbers = table.lookup(tf.constant(lines[0].split()))
with tf.Session() as sess:
  tf.tables_initializer().run()
  print "{} --> {}".format(lines[0], numbers.eval())   

1.2.1
118 words into vocab.tsv
This might be the best "taco" truck on the planet. Hidden between a smoke shop and The Home Depot, this semi permanent, stylish and clean cafe on wheels. --> [ 42  12  41 118  33 119  51  47 118 119  96  39 112  54   1  87 111  56
 119  81  19 119  23  87 117  99  47 119]


In [34]:
!head vocab.tsv

ZYXW
shop
just
cheap
go
Indiana
its
We
seems
laid


Sprawdzenie działania modelu na małym zbiorze danych

In [163]:
%bash
echo "bucket=${BUCKET}"
rm -rf outputdir
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
python -m trainer.task \
   --bucket=${BUCKET} \
   --output_dir=outputdir \
   --job-dir=./tmp --train_steps=1000

bucket=dswbiznesie
5687 words into gs://dswbiznesie/data/vocab_words
{'review_count': <tf.Tensor 'DecodeCSV:4' shape=(?, 1) dtype=float32>, 'avg_rating': <tf.Tensor 'DecodeCSV:5' shape=(?, 1) dtype=float32>, 'reviews': <tf.Tensor 'DecodeCSV:0' shape=(?, 1) dtype=string>, 'categories': <tf.Tensor 'DecodeCSV:2' shape=(?, 1) dtype=string>, 'zip_code': <tf.Tensor 'DecodeCSV:3' shape=(?, 1) dtype=string>} Tensor("hash_table_Lookup:0", shape=(?, 1), dtype=int64)
Tensor("string_to_index_1_Lookup:0", shape=(?, ?), dtype=int64)
words_sliced=SparseTensor(indices=Tensor("StringSplit:0", shape=(?, 2), dtype=int64), values=Tensor("StringSplit:1", shape=(?,), dtype=string), dense_shape=Tensor("StringSplit:2", shape=(2,), dtype=int64))
words_embed=Tensor("EmbedSequence/embedding_lookup:0", shape=(?, 77838, 10), dtype=float32)
words_conv=Tensor("Squeeze_1:0", shape=(?, 19460), dtype=float32)
embed_categories shape
(?, 100, 10)
categories_out shape
(?, 100, 1)
Tensor("DecodeCSV:5", shape=(?, 1), dtype=

Copying gs://dswbiznesie/data/train.csv...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc359cd3f90>, '_model_dir': 'outputdir/', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
2017-11-19 01:06:31

<h2> Trenowanie modelu w Cloud ML </h2>

Jak widać kod pythonowy działa przy bezpośrednim wywołaniu. Można przetestować go lokalnie w Cloud ML.

In [ ]:
%bash
rm -rf trained_model
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${REPO}/notebooks/trainer \
   -- \
   --output_dir=${REPO}/notebooks/trained_model \
   --bucket=${BUCKET} \
   --output_dir=outputdir \
   --job-dir=./tmp --train_steps=1000

<b> Wizualizacja procesu uczenia w Tensorboard </b>

In [13]:
from google.datalab.ml import TensorBoard
TensorBoard().start('{}/notebooks/outputdir'.format(REPO))

TensorBoard was started successfully with pid 5268. Click here to access it.

5268

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-11-11 18:00:48.963627. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [ ]:
from google.datalab.ml import TensorBoard
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Tak przygotowany kod można uruchomić w klastrze Cloud ML. Status joba uczącego znajduje się w konsoli GCP.

In [9]:
%bash
OUTDIR=gs://${BUCKET}/trained_model
JOBNAME=hygiene_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gsutil cp trainer/*.py $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=$(pwd)/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC --runtime-version=1.2 \
   -- \
   --bucket=${BUCKET} \
   --output_dir=${OUTDIR} \
   --train_steps=1000

gs://dswbiznesie/trained_model europe-west1 hygiene_171111_161537
jobId: hygiene_171111_161537
state: QUEUED


Removing gs://dswbiznesie/trained_model/__init__.py#1510416808672460...
Removing gs://dswbiznesie/trained_model/model.py#1510416808944571...
Removing gs://dswbiznesie/trained_model/task.py#1510416809177912...
/ [3/3 objects] 100% Done                                                       
Operation completed over 3 objects.                                              
Copying file://trainer/__init__.py [Content-Type=text/x-python]...
Copying file://trainer/model.py [Content-Type=text/x-python]...
Copying file://trainer/task.py [Content-Type=text/x-python]...
-
Operation completed over 3 objects/12.8 KiB.                                     
Job [hygiene_171111_161537] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe hygiene_171111_161537

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs hygiene_171111_161537
/usr/local/lib/python2.7/dist-packages/simplejson/e

<h2> Opublikowanie wytrenowanego modelu </h2>

In [ ]:
%bash
gsutil ls gs://${BUCKET}/trained_model/export/Servo/

In [ ]:
%bash
MODEL_NAME="hygiene"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/trained_model/export/Servo/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

<h2> Serwowanie predykcji </h2>

Opublikowany model, zostanie użyty do serwowania predykcji. Aby otrzymać predykcje, wystarczy wykonać JSONowy request do api predykcji.

In [ ]:
%bash
gcloud ml-engine predict --model=hygiene --version=v1 --json-instances=./passed.json

In [ ]:
%bash
gcloud ml-engine predict --model=hygiene --version=v1 --json-instances=./not-passed.json

In [ ]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1beta1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1beta1_discovery.json')

request_data = {"instances":
  [
      {
      "reviews": preprocess_text(" Great food, friendly staff, go go go go go! :) Ah, we always go here everytime we go to Pike Place. I always get either the Halibut sandwich or the Halibut platter.The sandwich comes in a french bread from Le Panier, delicious. The fish is fresh and why wouldn't it be? The Market Grill is inside pike place across from all the fish vendors! The cook takes it out and grills it in front of you with an option to add the blackening seasoning. I always get that. The platter comes with garlic bread, organic brown rice and organic salad. The rice is slightly flavored from the delicious dressing they have from the salad and it's soooo good. The fish is also very good.I've also had their clam chowder and I like it also.Ah, maybe I'll stop by this weekend. It's also a fun atmosphere because you can sit on the stools and watch people as you enjoy your fish. I've recommended this place to everyone I know! Especially if they're looking for something quick, cheaper and delicious in Pike Place. I've tried almost all of their sandwiches and I can honestly say they have been great. I've yet to try the chicken sandwich, but when your other options are salmon, halibut and prawns it's kind of hard to do so. It's totally worth grabbing the sandwich and walking around the market for a spot on a bench since the stools in their nook are almost always packed. I think there's only 8 of them. The salmon is probably my favorite but you really can't go wrong here. The chowder is also worth the wait. Make sure to treat yourself to the sides of chowder and coleslaw. You won't be disappointed. The staff here is usually very funny and talkative. They'll make you feel right at home, carrying on conversation while flipping your food on the grill just a few feet away from the counter. I bring my friends here whenever someone comes to visit and they all love it too. Just got back from a short mid-winter stay in Seattle. &#160;The weather was beautiful and we walked down to the Market for some lunch armed with several Yelp recommendations.We ended up at the Market Grill. &#160;Of course the place was crowded. &#160;We could have waited a few and grabbed one of the stools but opted to order take-out and eat in the park.In spite of some of the reviews indicating the sandwiches were a bit small, they looked big enough for us to share. &#160;We went with the salmon, blackened, and loaded with the onions, and rosmary mayo, lettuce, and tomatoes, two sides of slaw, and a medium chowder to share and headed to the park to eat our lunch. &#160;Total price $20.00I'm telling you, this is a damn good sandwich. &#160;Every component worked together from the crunchy baguette, the perfect tomatoes, the crisp lettuce, the MAYO (yum), and of course the huge piece of fresh perfectly cooked fish. &#160;The Slaw was crisp, chunky, and very flavorful, with the right amount of heat. &#160;The chowder was also very good. &#160;Definitely home made. &#160;Possibly a bit light on the clams, but then again, we finished the whole thing and were basically licking the bowl.We will come back - for sure.")
       },
    {
    "reviews":" Pam's is really great. &#160;It's simple and delicious food that was perfect for a cold Seattle night that needed a little spicing up. &#160;I've only had the beef roti &#160;but I was so full that I couldn't fit any appetizer or dessert which I usually have room for. The beef roti was filled with curried chick peas, potatoes, large chunks of beef, and a yummy sauce. &#160;I can't wait to go back and try the other menu items. Also, A+ for service that was efficient, kind, and not overwhelming! Despite it's unassuming storefront facade, this is a very impressive little restaurant. &#160;Two of us tried two different dishes - lamb roti and a brown rice with roast peas - and both were perfect. For dessert we had an unusual and lovely little cake of coconut and casava root. The menu is simple, but every dish is excellent. &#160;It can't be easy to combine so many different influences (Carribean, African, East Indian) and still have the food taste so good and so subtle. Plus, on the rainy day that I was there, the mix of tropical colors, warm smells and Carribean music were a welcome change of atmosphere. I'm looking forward to going back. Really good food. When I ate meat, I tried both the chicken and beef. Both were really good. Now that I'm a vegetarian, I just eat the potato and chickpea mix and that is really good too. I've tried both styles of roti (wrapped up like a burrito and on the side like a big tortilla) and both are really good. The habanero hot sauce is amazing. I love it! Service is good too. I just wish it was closer to my apartment so I could eat there more often. The menu is limited, but what they do have is really good. They have perfected a small menu, which I think is much better than having a big mediocre menu. I find myself in Pam's Kitchen at least once per month, and am continuously amazed the place is not packed with people. &#160;The food is uniquely delicious and always exceptionally prepared - I usually order a Chicken Roti (Roti = chick peas potatoes and onions baked together with choice of meat and powerful blend of Caribbean spice, then wrapped in savory pastry) and one of their strangely addicting milk based punches, which come in peanut, carrot and (my favorite) pumpkin. &#160;At $15 for those two items (including tax and tip) it is a pretty good deal, though i realize you could get two mediocre teryaki meals from Tokyo Garden for the roughly the same price. &#160;They have a good selection of beer to fit the menu, service is great, and the island atmosphere is undeniably cool. &#160; Hungry? &#160;Thats good - I've never left Pam's feeling less than comfortably stuffed. &#160;The place is perfect for groups of three or four, so bring some friends and impress them with a creative and international dining experience on a student's budget. This place was fantastic! &#160;The owners were very nice and attentive--he even pointed out the Yelp! sticker on his window--what a business man!My friend and I decided this place looked interesting and we'd never had this kind of food before. &#160;Basically, we came looking for adventure and left satisfied. &#160;We split the Lamb Roti and were incredibly full. &#160;My friend thought it should be more saucy, but I was really happy w/it. &#160; &#160;Plus, the music is fantastic. &#160;At first I commented that it was way too loud, but then I kept dancing in my chair. &#160;It reminded me of some cheesy Bollywood-esque score. I'll admit that I'm naive about roti. &#160;So, I must make two disclaimers.First, I have no idea whether I'm being ripped off. &#160;I've read that, by island standards, this is way-expensive. &#160;Then again, this is Seattle, and living costs tend to make *everything* more expensive than on the streets of Chaguanas. &#160;It's not a steal (by even Seattle standards), but it also doesn't seem to be excessive.Second, I don't know how this holds up to roti in the Caribbean. &#160;It's hard to imagine that flaky goodness gets any flakier or goodness-er than this, but I hope it doesn't, because I fear for my carbohydrate intake once something better opens up.As you can probably already tell, I enjoyed Pam's Kitchen. &#160;I had the goat roti, with the version where some assembly is required (as opposed to the pre-filled roti, this one comes with all components separated). &#160;The goat was tender and reasonably juicy, and made for a perfect filling. &#160;I also got a spicy pumpkin side ($3), which was exactly as advertised, and a good complement to the main dish.Service for me was, contrary to some reviews, exemplary. &#160;They go by a sit-where-you-like system, but I was addressed promptly. &#160;I was also entertained by the cook briefly, who explained the positive medicinal effects of roti. &#160;I'm not sure if roti is responsible for curing all of the listed diseases, but if it is, I demand U.N. funding immediately, straight to my house.As mentioned, prices are moderate. &#160;Roti will run you $8 to $10, which seems high for the University District, but is about what this sort of food would run you elsewhere in town. &#160;Certainly, not enough to keep me away, but probably enough to deny me the prophylactic powers of Pam's delicacies."
      },
      {
        "reviews":" OMG the spicy Teriyaki almost killed me today. &#160;I order it all the time and love it but this time it was like eating a fireball! &#160;I called to complain and they said \"next time we make not spicy\" &#160;What the heck? &#160;Also, someone please tell the cashier dude to clean his finger nails... Grosssssssss Hmmm should I go to Subway like always or try something new? Eh I might as well try something new. However, my first glance inside Teriyaki Plus isn't too encouraging as I don't see a soul inside. Eh, there are always rushes and slow periods. I go inside since there is no menu outside to look at. Immediately the cashier comes out to stare at me as if guilting me into buying something, instead of walking out after seeing the interior (dilapidated with dirty floors, and walls, along with tables and chairs that clearly need to be replaced). I order the white chicken and beef combo so that I can try each meat (in case one isn't very good). After waiting about 15 minutes (and getting annoyed because I got takeout since I'm in a hurry and yet have been waiting as long as I would have at a normal restaurant). The cashier comes out with my food and asks \"Fork?\" while motioning eating with a fork. I left with chopsticks (yes I'm a white girl who can use chopsticks) and my food in a bag. Upon opening the bag I noticed they used compostable containers which is nice, however the dressing from the salad had leaked into my bag. I opened the container and realized they have given me a TON of salad drenched in an overly sweet creamy sauce. Maybe they thought I needed more vegetables in my diet? While they also gave me a lot of rice, the meat was lacking compared to most teriyaki places. Which is fine as long as the meat is quality. However, the beef was barely tolerable and the chicken was even drier than the beef. Did they perhaps forget to add sauce? I never drench my food in anything, but this called for drastic measures. After drenching all the meat in soy sauce it was slight less dehydrated, but still as tough and chewy as before. My conclusion: how exactly is this place still open? Is it due to stupid people like me who don't check Yelp when they're starving and in a hurry? Well, back to good old Subway for me in times of desperate takeout (or Qdoba across the street)."
      },
      {
        "reviews":" Nothing is fresh about it! The quality of food is lower then the worst fast food place you've ever been to (and the smell is awful). Its dirty and If you are used to eating somewhat healthy you will feel sick. The reason this place has any customers at all is because for three 20+ stores office buildings there are only 4 places to eat. The way the food is made grosses me out; when I watch things being slapped on top of each other I wish I had no visibility into their kitchen. On top of that because people have limited options they charge a lot for what they offer."
      },
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'hygiene', 'v1')
response = api.projects().predict(body=request_data, name=parent).execute()
print "response={0}".format(response)